In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
output = 'radio_data.npz'

f = np.load(output)


In [5]:
n_train  = 40000

x_train,x_test = f['traces'][:n_train],f['traces'][n_train:]
signals = f['signals']
labels = (signals.std(axis=-1) != 0).astype(float)  # define training label (1=cosmic event, 0=noise)
y_train, y_test = labels[:n_train], labels[n_train:]

In [6]:
sigma = x_train.std()
x_train /= sigma
x_test /= sigma

In [7]:
print(x_train.shape)
print(y_train.shape)

(40000, 500)
(40000,)


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_layers, dropout_rate, output_size, sequence_length):
        super(BiLSTM, self).__init__()
        
        # Hidden dimensions
        self.hidden_sizes = hidden_sizes
        self.num_layers = num_layers

        # Number of hidden layers and bidirectionality
        self.lstm1 = nn.LSTM(input_size, hidden_sizes[0], num_layers, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(hidden_sizes[0]*2, hidden_sizes[1], num_layers, batch_first=True, bidirectional=True)
        self.lstm3 = nn.LSTM(hidden_sizes[1]*2, hidden_sizes[2], num_layers, batch_first=True, bidirectional=True)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        
        # Calculate feature size
        self.feature_size = sequence_length * hidden_sizes[2] * 2
        
        # Fully connected layer
        self.fc = nn.Linear(self.feature_size, output_size)
        
    def forward(self, x):
        
        # Passing in the input and hidden state into the model and obtaining outputs
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        x = x.contiguous().view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        x = torch.sigmoid(x)
        
        return x

sequence_length = 500  # Length of your sequences
input_size = 1  # Number of features
hidden_sizes = [32, 64, 10]  # Number of hidden units in the LSTM layers
num_layers = 1  # Number of LSTM layers
dropout_rate = 0.3  # Dropout rate after the final LSTM layer
output_size = 1  # Number of output units

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BiLSTM(input_size, hidden_sizes, num_layers, dropout_rate, output_size, sequence_length).to(device)
print(model)


BiLSTM(
  (lstm1): LSTM(1, 32, batch_first=True, bidirectional=True)
  (lstm2): LSTM(64, 64, batch_first=True, bidirectional=True)
  (lstm3): LSTM(128, 10, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=10000, out_features=1, bias=True)
)


In [23]:
class model2(nn.Module):
    def __init__(self):
        super(model2, self).__init__()

        # Layers
        self.LSTM_b_1 = nn.LSTM(1, 32, 1, batch_first=True, bidirectional=True)
        self.LSTM_b_2 = nn.LSTM(64, 64, 1, batch_first=True, bidirectional=True)
        self.LSTM_b_3 = nn.LSTM(128, 10, 1, batch_first=True, bidirectional=True)
        
        self.FC = nn.Linear(500*20, 1)

    def forward(self, x):
        out, _ = self.LSTM_b_1(x)
        out, _ = self.LSTM_b_2(out)
        out, _ = self.LSTM_b_3(out)
        out = out.contiguous().view(out.size(0), -1)  # Flatten
        out = F.dropout(out, p=0.3)  # Dropout
        out = self.FC(out)
        out = torch.sigmoid(out)
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model2().to(device)

In [25]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
# Prepare data
x_train_torch = torch.tensor(x_train[..., np.newaxis], dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32)

# Split train and validation data
x_train_torch, x_val_torch, y_train_torch, y_val_torch = train_test_split(x_train_torch, y_train_torch, test_size=0.1)


train_loader = DataLoader(TensorDataset(x_train_torch, y_train_torch), batch_size=256, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val_torch, y_val_torch), batch_size=256, shuffle=False)

# Define model, optimizer and loss function

optimizer = Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()

# Define learning rate scheduler and early stopping parameters
scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=5, verbose=True, min_lr=1e-5)
n_epochs_stop = 15
epochs_no_improve = 0
min_val_loss = np.Inf

val_losses = []
train_losses = []


# Training loop
for epoch in range(100):  # loop over the dataset multiple times
    running_loss = 0.0
    model.train()  # put model in training mode
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    # Normalizing the loss by the total number of train batches
    running_loss /= len(x_train_torch)
    # print(f'Training Loss: {running_loss}')

    # Calculate validation loss
    val_loss = 0.0
    model.eval()  # put model in evaluation mode
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels.unsqueeze(1)).item()

    # Normalizing the loss by the total number of val batches
    val_loss /= len(x_val_torch)
    # print(f'Validation Loss: {val_loss}')
    
    # Step the scheduler
    scheduler.step(val_loss)
    val_losses.append(val_loss)
    train_losses.append(running_loss)

    print(f'Epoch: {epoch+1} \tTraining Loss: {running_loss} \tValidation Loss: {val_loss}')

print('Finished Training')


Epoch: 1 	Training Loss: 0.0019955048221680853 	Validation Loss: 0.0019877799078822137
Epoch: 2 	Training Loss: 0.0019695439661542575 	Validation Loss: 0.0020250103026628495
Epoch: 3 	Training Loss: 0.0019646609922250114 	Validation Loss: 0.0020366621538996698
Epoch: 4 	Training Loss: 0.001954938246972031 	Validation Loss: 0.002070267729461193
Epoch: 5 	Training Loss: 0.0019440842121839523 	Validation Loss: 0.002072449065744877
Epoch: 6 	Training Loss: 0.0019392280826965968 	Validation Loss: 0.0020970431193709374
Epoch 00007: reducing learning rate of group 0 to 5.0000e-04.
Epoch: 7 	Training Loss: 0.0019357089259558254 	Validation Loss: 0.0021138175055384636
Epoch: 8 	Training Loss: 0.001882757211724917 	Validation Loss: 0.002131369970738888
Epoch: 9 	Training Loss: 0.0018729489834772217 	Validation Loss: 0.0021407899037003517
Epoch: 10 	Training Loss: 0.0018762283482485348 	Validation Loss: 0.0021508644968271256
Epoch: 11 	Training Loss: 0.0018630236362417538 	Validation Loss: 0.0021